In [1]:
import pdb
import requests
from bs4 import BeautifulSoup   

In [2]:
response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')

In [3]:
print(response)

<Response [200]>


In [4]:
response.status_code

200

Адрес, по которому мы обращались, возвращает результат в json формате. Эти данные уже лежат в атрибуте text в полученном ответе response:

In [5]:
print(response.text)

{
    "Date": "2021-05-08T11:30:00+03:00",
    "PreviousDate": "2021-05-07T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2021\/05\/07\/daily_json.js",
    "Timestamp": "2021-05-10T20:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 57.6418,
            "Previous": 57.7524
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 43.6358,
            "Previous": 43.8946
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 103.2288,
            "Previous": 103.6695
 

Сейчас текст хранится просто в строковой переменной. Далее мы можем превратить эту строку в словарь. Сделать это можно с помощью JSON-парсера python, либо воспользовавшись методом json, который уже встроен в объект ответа response:

In [6]:
currences = response.json()

In [7]:
currences

{'Date': '2021-05-08T11:30:00+03:00',
 'PreviousDate': '2021-05-07T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2021/05/07/daily_json.js',
 'Timestamp': '2021-05-10T20:00:00+03:00',
 'Valute': {'AUD': {'ID': 'R01010',
   'NumCode': '036',
   'CharCode': 'AUD',
   'Nominal': 1,
   'Name': 'Австралийский доллар',
   'Value': 57.6418,
   'Previous': 57.7524},
  'AZN': {'ID': 'R01020A',
   'NumCode': '944',
   'CharCode': 'AZN',
   'Nominal': 1,
   'Name': 'Азербайджанский манат',
   'Value': 43.6358,
   'Previous': 43.8946},
  'GBP': {'ID': 'R01035',
   'NumCode': '826',
   'CharCode': 'GBP',
   'Nominal': 1,
   'Name': 'Фунт стерлингов Соединенного королевства',
   'Value': 103.2288,
   'Previous': 103.6695},
  'AMD': {'ID': 'R01060',
   'NumCode': '051',
   'CharCode': 'AMD',
   'Nominal': 100,
   'Name': 'Армянских драмов',
   'Value': 14.2175,
   'Previous': 14.3175},
  'BYN': {'ID': 'R01090B',
   'NumCode': '933',
   'CharCode': 'BYN',
   'Nominal': 1,
   'Name': 

In [8]:
currences['Valute']['CZK']['Name']

'Чешских крон'

Оформляем функцию
В завершение давайте оформим наши вычисления в отдельную функцию, которой будет удобно пользоваться. На вход она должна принимать два параметра:

1. Название валюты currency. Например, 'EUR' или 'USD'.

2. Формат ответа format. При значении 'full' будем отдавать все, что знаем о валюте. Например, для currency = 'USD':

In [9]:
def exchange_rates(currency, format='full'):    
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']    
    data = response[currency]     
    if format == 'full':    
        return data      
    elif format == 'value':    
        return data['Value']    

Напишите функцию currency_name, которая по ID валюты возвращает ее название на русском языке.

In [10]:
def currency_name(id_):
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']
#     pdb.set_trace()
    for valute in response:
        if response[valute]['ID'] == id_:
            return response[valute]['Name']
        
    


In [11]:
currency_name('R01700J')

'Турецких лир'

# HTML-страницы

Довольно часто приходится добывать информацию не из удобно форматированного json-файла, а прямо с HTML-страниц. Получить содержимое страницы в большинстве случаев несложно, труднее извлечь из HTML-кода нужную информацию. В качестве примера мы рассмотрим страницу новости, из которой будем доставать полезную информацию: 1) заголовок страницы; 2) дату публикации; 3) текст публикации; 4) ссылки на странице.

In [12]:
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
  
response = requests.get(url)  
print(response.status_code)
print(response.text)

200
<!doctype html>
<html class="no-js bg-fixed _no-bg" style="background-image:url(https://nplus1.ru/images/2019/06/04/b32b62189fb87cce895e229e1d6d27b4.jpeg)" lang="">
<head>
    
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
                <meta name='wmail-verification' content='7991d7eb02d759f05b9050e111a7e3eb' />

    <link rel="canonical" href="https://nplus1.ru/news/2019/06/04/slothbot" />

        <title>Робота-ленивца научили лазать по паутине из тросов</title>

    	    <meta itemprop="datePublished" content="2019-06-04"/>
	
	    <meta name="mediator_author" content="Григорий Копиев"/> 
	
        <!-- amp page -->
    <link rel="amphtml" href="https://nplus1.ru/news/2019/06/04/slothbot/amp">
    

        <!-- for Google -->
     

# Библиотека BeautifulSoup

In [13]:
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
response = requests.get(url)  

page = BeautifulSoup(response.text, 'html.parser')

In [14]:
page.title.text

'Робота-ленивца научили лазать по паутине из тросов'

# Данные со страницы

Напишите функцию wiki_header, которая по адресу страницы возвращает заголовок для статей на википедии

In [15]:
def wiki_header(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.find('h1').text

In [16]:
wiki_header('https://en.wikipedia.org/wiki/Operating_system')

'Operating system'

У вас есть переменная page, в которой хранится содержимое html-страницы. На странице есть элемент в тэге span, у которого атрибут id равен 'target'. Напишите строчку кода, которая присвоит текст этого элемента переменной value.

Запишите ваш код в одну строку без пробелов. Используйте апострофы для передачи параметров, содержащих значение тэга и его id.

In [17]:
# value=page.find('span',class='target')

# Сбор нескольких элементов

Рассмотрим случай, когда нам нужно сразу много элементов. Пусть мы хотим получить название всех ссылок на странице в википедии про языки программирования.

Для ссылок существует тэг <a></a>.  Давайте попробуем использовать find  

In [18]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'  
  
response = requests.get(url)  
page = BeautifulSoup(response.text, 'html.parser')  
page.find('a')  

<a id="top"></a>

Метод find возвращает только первый подходящий элемент. Если нам надо их больше, нужно воспользоваться методом find_all



In [19]:
links = page.find_all('a')  
# Посмотрим, сколько всего мы получили  
print(len(links))

927


In [20]:
# Посмотрим на некоторые из ссылок  
print([link.text for link in links[500:510]])  

['Orwell', 'Oxygene', 'Oz', 'edit', 'P', 'P4', 'P′′', 'ParaSail (programming language)', 'PARI/GP', 'Pascal']


Ещё одна полезная вещь:  последовательный поиск, т.е. мы можем найти сначала один элемент, а потом сделать внутри него второй поиск. Давайте выведем названия всех ссылок для языков программирования, которые начинаются на литеру "A".

In [21]:
# Получаем все элементы с тегом 'div' и классом 'div-col'  
all_blocks = page.find_all('div', class_='div-col')  
  
# Выбираем первый по счету блок  
first_block = all_blocks[0]  
# Берём оттуда ссылки (ограничимся первыми десятью)
links = first_block.find_all('a')
print([link.text for link in links[:10]])

['A.NET (A#/A sharp)', 'A-0 System', 'A+ (A (plus)', 'ABAP', 'ABC', 'ABC ALGOL', 'ACC', 'Accent (Rational Synergy)', 'Ace DASL (Distributed Application Specification Language)', 'Action!']


# Напишите функцию get_actors, которая по ссылке на страницу фильма на кинопоиске возвращает список актёров из колонки справа.



In [22]:
def get_actors(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    blocks = page.find_all('ul', class_='styles_list__I97eu')
#     actors = blocks.find('li')
    return blocks

In [23]:
get_actors('https://www.kinopoisk.ru/film/42326/')

[]

# HTML-таблицы

Метод read_html из pandas умеет автоматически находить на HTML-странице таблицы и возвращать их списком из датафреймов:

In [24]:
import pandas as pd  
url = 'https://www.cbr.ru/key-indicators/'  
pd.read_html(url)[0]

,0,1,2
0,валюта,07.05.2021,08.05.2021
1,Доллар США USD,745770,741373
2,Евро EUR,896788,895060


Притворяемся человеком

In [25]:
from bs4 import BeautifulSoup  
import pandas as pd  
import requests  
  
url = 'https://www.cbr.ru/key-indicators/'
soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')  

In [26]:
all_blocks = soup.find_all('div', class_='key-indicator_content offset-md-2')
data = all_blocks[1].find('table') 

In [27]:
df = pd.read_html(str(data))[0]

In [28]:
df

,0,1,2
0,рублей за грамм,07.05.2021,08.05.2021
1,Золото Au,"4 299,44","4 339,29"
2,Серебро Ag,6408,6525
3,Платина Pt,"2 961,16","2 979,46"
4,Палладий Pd,"7 061,24","6 981,47"


Напишите программу, которая забирает данные из таблицы рейтинга банков с https://www.banki.ru/banks/ratings/, делает из него датафрейм и сохраняет его в переменую df.

In [29]:
url = 'https://www.banki.ru/banks/ratings/'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
page = BeautifulSoup(response.text, 'html.parser')

blocks = page.find_all('div', class_='layout-column-full')
table = blocks[2].find('table', attrs={'data-test': 'rating-table',
              'class': 'standard-table standard-table--row-highlight margin-bottom-small margin-top-x-small', }, )

df = pd.read_html(str(table))[0]

In [30]:
df

место                  название банка лицензия №, Регион  \
    место                  название банка лицензия №, Регион   
0       1            СберБанк лицензия № 1481, Москва и обл.   
1       2        ВТБ лицензия № 1000, Санкт-Петербург и обл.   
2       3          Газпромбанк лицензия № 354, Москва и обл.   
3       4  Национальный Клиринговый Центр лицензия № 3466...   
4       5          Альфа-Банк лицензия № 1326, Москва и обл.   
5       6      Россельхозбанк лицензия № 3349, Москва и обл.   
6       7  Московский Кредитный Банк лицензия № 1978, Мос...   
7       8       Банк Открытие лицензия № 2209, Москва и обл.   
8       9        Совкомбанк лицензия № 963, Костромская обл.   
9      10      Райффайзенбанк лицензия № 3292, Москва и обл.   
10     11             Росбанк лицензия № 2272, Москва и обл.   
11     12         ЮниКредит Банк лицензия № 1, Москва и обл.   
12     13               Траст лицензия № 3279, Москва и обл.   
13     14      Россия лицензия № 328, Санкт-Петербург и обл.   
14     15  Всероссийский Банк Развития Регионов лицензия ...   
15     16       Тинькофф Банк лицензия № 2673, Москва и обл.   
16     17  Санкт-Петербург лицензия № 436, Санкт-Петербур...   
17     18            Ситибанк лицензия № 2557, Москва и обл.   
18  19 +2         Банк ДОМ.РФ лицензия № 2312, Москва и обл.   
19     20         Новикомбанк лицензия № 2546, Москва и обл.   
20  21 +1            СМП Банк лицензия № 3368, Москва и обл.   
21  22 −3                 Ак Барс лицензия № 2590, Татарстан   
22     23          Банк Уралсиб лицензия № 30, Москва и обл.   
23     24             БМ-Банк лицензия № 2748, Москва и обл.   
24     25           Почта Банк лицензия № 650, Москва и обл.   
25     26  Московский Областной Банк лицензия № 1751, Мос...   
26  27 +1            Пересвет лицензия № 2110, Москва и обл.   
27  28 +1  Московский Индустриальный Банк лицензия № 912,...   
28  29 +1    Русский Стандарт лицензия № 2289, Москва и обл.   
29  30 −3  Национальный Расчетный Депозитарий лицензия № ...   
30  31 +1                  РНКБ лицензия № 1354, Симферополь   
31  32 −1  Уральский Банк Реконструкции и Развития лиценз...   
32     33        Абсолют Банк лицензия № 2306, Москва и обл.   
33     34     Хоум Кредит Банк лицензия № 316, Москва и обл.   
34     35          Банк Зенит лицензия № 3255, Москва и обл.   
35     36      Восточный Банк лицензия № 1460, Амурская обл.   
36     37            МТС Банк лицензия № 2268, Москва и обл.   
37     38    Транскапиталбанк лицензия № 2210, Москва и обл.   
38     39        Росэксимбанк лицензия № 2790, Москва и обл.   
39     40      Инвестторгбанк лицензия № 2763, Москва и обл.   
40     41            ИНГ Банк лицензия № 2495, Москва и обл.   
41     42           Локо-Банк лицензия № 2707, Москва и обл.   
42  43 +2        Сетелем Банк лицензия № 2168, Москва и обл.   
43  44 −1    Ренессанс Кредит лицензия № 3354, Москва и обл.   
44  45 +2            ОТП Банк лицензия № 2766, Москва и обл.   
45  46 +3        ВУЗ-Банк лицензия № 1557, Свердловская обл.   
46  47 +1            МСП Банк лицензия № 3340, Москва и обл.   
47  48 +2            Авангард лицензия № 2879, Москва и обл.   
48  49 −3         Возрождение лицензия № 1439, Москва и обл.   
49  50 +1  Азиатско-Тихоокеанский Банк лицензия № 1810, А...   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [31]:
table.prettify

<bound method Tag.prettify of <table class="standard-table standard-table--row-highlight margin-bottom-small margin-top-x-small" data-test="rating-table">
<thead>
<tr>
<th class="table-title th-sortable th-sortable--asc" rowspan="2">
<a class="th-sortable__text th-sortable__text--double-height" href="/banks/ratings/?sort_param=rating&amp;sort_order=DESC">
                        место                    </a>
</th>
<th class="table-title th-sortable" rowspan="2">
<a class="th-sortable__text th-sortable__text--double-height" href="/banks/ratings/?sort_param=bankname&amp;sort_order=ASC">
                        название банка                        <div class="font-size-small th-sortable__text-sub font-normal">лицензия №, Регион</div>
</a>
</th>
<th class="table-title is-center" colspan="2" width="20%">
                    показатель,
                    тыс. рублей                                                        </th>
<th class="table-title colspan is-center" colspan="2">изменение

# Запрос к API из кода

In [32]:
import requests  
token = '669f554d669f554d669f554d0366e89c596669f669f554d06070c11afa2a9471230bd66'
url = 'https://api.vk.com/method/users.get'   
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'}  
  
# Мы можем выставить параметры запроса через аргумент params  
response = requests.get(url, params=params)  
response.text  

'{"response":[{"first_name":"Павел","id":1,"last_name":"Дуров","can_access_closed":true,"is_closed":false,"sex":2,"bdate":"10.10.1984"}]}'

Мы получили строку в формате JSON. Как мы помним по первому разделу, её можно преобразовать в словарь методом json и после этого обращаться к различным полям. Кроме того, такие большие вложенные словари нагляднее выводить с помощью функции pprint (~pretty print, красивый вывод), которым мы и воспользуемся

In [33]:
from pprint import pprint  
  
pprint(response.json())  

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


In [34]:
user = response.json()['response'][0]  

In [35]:
user['first_name']

'Павел'

Данный метод позволяет запрашивать сразу много пользователей (до 1000).  Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате, например: '1,2,3'.

In [36]:
ids = ",".join(map(str, range(1, 4)))  
print(ids)  

1,2,3


In [37]:
params = {'user_ids': ids, 'v': 5.95, 'fields': ['bday','country'], 'access_token': token, 'lang': 'ru'}  

In [38]:
pprint(requests.get(url, params=params).json()) 

{'response': [{'can_access_closed': True,
               'country': {'id': 1, 'title': 'Россия'},
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


# Задание 1 - Гендерный баланс

Используя API, определите долю женщин(sex=1) среди пользователей с id от 1 до 500. Иногда вам будут попадать пользователи, у которых пол не указан (sex=0), их не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после запятой, например, 0.55

Пример: если у нас будет 300 пользователей sex=1, 100 пользователей с sex=2 и 100 пользователей с sex=0, то в ответе должно быть 0.75

In [39]:
ids = ','.join(map(str,range(1, 501)))

In [40]:
token = '669f554d669f554d669f554d0366e89c596669f669f554d06070c11afa2a9471230bd66'
url = 'https://api.vk.com/method/users.get'
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex,bdate',
          'access_token': token, 'lang': 'ru'}

In [41]:
response = requests.get(url, params=params)
response = response.json()

In [42]:
pprint(response)

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова',
               'sex': 1},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 4,
               'last_name': '',
               'sex': 0},
              {'bdate': '18.11',
               'can_access_closed': True,
               'first_name': 'Илья',
               'id': 5,
               'is_closed': False,
               'last_name': 'Перекопский',
       

               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 127,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 128,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 129,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 130,
               'last_name': '',
               'sex': 0},
              {'bdate': '28.3.1984',
               'can_access_closed': False,
               'first_name': 'Шурикен',
               'id': 131,
               'is_closed': True,
               'last_name': 'Картошкин',
               'sex': 2},
              {'deactivated': 'deleted',
               'first_name

               'last_name': 'Журавлёв',
               'sex': 2},
              {'bdate': '15.8',
               'can_access_closed': False,
               'first_name': 'Татьяна',
               'id': 238,
               'is_closed': True,
               'last_name': 'Тимофеева',
               'sex': 1},
              {'bdate': '26.12',
               'can_access_closed': True,
               'first_name': 'Светлана',
               'id': 239,
               'is_closed': False,
               'last_name': 'Степанова',
               'sex': 1},
              {'deactivated': 'banned',
               'first_name': 'Влад',
               'id': 240,
               'last_name': 'Коробейников',
               'sex': 2},
              {'bdate': '19.8',
               'can_access_closed': True,
               'first_name': 'Наташа',
               'id': 241,
               'is_closed': False,
               'last_name': 'Власова',
               'sex': 1},
              {'bdate': '15.8.1983',

               'id': 346,
               'is_closed': True,
               'last_name': 'Маркина',
               'sex': 1},
              {'deactivated': 'banned',
               'first_name': 'Андрей',
               'id': 347,
               'last_name': 'Бойко',
               'sex': 2},
              {'deactivated': 'banned',
               'first_name': 'Галина',
               'id': 348,
               'last_name': 'Румянцева',
               'sex': 1},
              {'deactivated': 'banned',
               'first_name': 'Анастасия',
               'id': 349,
               'last_name': 'Силюкова',
               'sex': 1},
              {'deactivated': 'banned',
               'first_name': 'Павел',
               'id': 350,
               'last_name': 'Таразевич',
               'sex': 2},
              {'bdate': '16.8',
               'can_access_closed': True,
               'first_name': 'Елена',
               'id': 351,
               'is_closed': False,
               'l

               'last_name': 'Шехтер',
               'sex': 2},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 456,
               'last_name': '',
               'sex': 0},
              {'can_access_closed': True,
               'first_name': 'Владимир',
               'id': 457,
               'is_closed': False,
               'last_name': 'Баранов',
               'sex': 2},
              {'can_access_closed': True,
               'first_name': 'Сергей',
               'id': 458,
               'is_closed': False,
               'last_name': 'Нагапетян',
               'sex': 2},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 459,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'banned',
               'first_name': 'Ilya',
               'id': 460,
               'last_name': 'Dneprovsky',
               'sex': 2},
              {'d

In [44]:
len(response['response'])

500

In [45]:
fem_count = 0
lol_count = 0
for user in range(0,len(response['response'])):
    if response['response'][user]['sex'] == 1:
        fem_count += 1
    elif response['response'][user]['sex'] == 0:
        lol_count += 1
round(fem_count/(len(response['response']) - lol_count), 2)

0.48

# Ограничения API

**1 ограничение на количество вызовов в единицу времени;**
**2 ограничение на количество выгружаемых строк за один запрос.**

In [46]:
import requests  
url = 'https://api.vk.com/method/groups.getMembers'  
params = {  
    'group_id': 'vk',  
    'v': 5.95,  
    'access_token': token  
}  
response = requests.get(url, params = params)  
data = response.json()  
print(data)  

{'response': {'count': 11533306, 'items': [5, 6, 10, 19, 34, 47, 54, 79, 177, 193, 198, 205, 219, 239, 243, 254, 345, 404, 406, 407, 467, 485, 510, 550, 619, 628, 640, 643, 690, 702, 720, 721, 724, 744, 804, 809, 831, 832, 847, 900, 905, 907, 914, 943, 952, 958, 966, 976, 979, 1000, 1018, 1023, 1032, 1033, 1038, 1039, 1059, 1097, 1131, 1139, 1140, 1159, 1174, 1188, 1290, 1301, 1333, 1334, 1336, 1351, 1359, 1386, 1388, 1406, 1411, 1418, 1432, 1494, 1500, 1531, 1543, 1568, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1650, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1754, 1796, 1814, 1820, 1829, 1834, 1839, 1840, 1843, 1858, 1863, 1868, 1869, 1889, 1917, 1943, 1947, 1955, 1969, 2019, 2028, 2050, 2051, 2052, 2059, 2077, 2103, 2145, 2150, 2195, 2201, 2202, 2230, 2236, 2273, 2281, 2294, 2296, 2298, 2376, 2389, 2395, 2403, 2412, 2436, 2456, 2466, 2470, 2484, 2515, 2527, 2539, 2571, 2576, 2592, 2601, 2622, 2644, 2654, 2692, 2706, 2745, 2755, 2767, 2787, 2797, 2827, 2858, 2896, 2909, 29

In [47]:
len(data['response']['items'])  

1000

Давайте выведем на экран первые 20 пользователей из нашей первой попытки с 1000 пользователей, чтобы было с чем сверить результат выгрузки из 20 пользователей:

In [48]:
users_for_checking = data['response']['items'][:20]  
print(users_for_checking)  

[5, 6, 10, 19, 34, 47, 54, 79, 177, 193, 198, 205, 219, 239, 243, 254, 345, 404, 406, 407]


Теперь используем count и offset, чтобы получить те же id по 5 за раз

In [60]:
count = 1000
offset = 0  
user_ids = []
max_count = 99999
while offset < max_count:  
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    params = {  
        'group_id': 'vk',  
        'v': 5.95,  
        'count': count,  
        'offset': offset,  
        'access_token': token  
    }     
    # такой же запрос как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']   
  
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  
      
# Выгружаю 5 пользователей с offset = 0  
# Выгружаю 5 пользователей с offset = 5  
# Выгружаю 5 пользователей с offset = 10  
# Выгружаю 5 пользователей с offset = 15  
print(user_ids)  

Выгружаю 1000 пользователей с offset = 0
Выгружаю 1000 пользователей с offset = 1000
Выгружаю 1000 пользователей с offset = 2000
Выгружаю 1000 пользователей с offset = 3000
Выгружаю 1000 пользователей с offset = 4000
Выгружаю 1000 пользователей с offset = 5000
Выгружаю 1000 пользователей с offset = 6000
Выгружаю 1000 пользователей с offset = 7000
Выгружаю 1000 пользователей с offset = 8000
Выгружаю 1000 пользователей с offset = 9000
Выгружаю 1000 пользователей с offset = 10000
Выгружаю 1000 пользователей с offset = 11000
Выгружаю 1000 пользователей с offset = 12000
Выгружаю 1000 пользователей с offset = 13000
Выгружаю 1000 пользователей с offset = 14000
Выгружаю 1000 пользователей с offset = 15000
Выгружаю 1000 пользователей с offset = 16000
Выгружаю 1000 пользователей с offset = 17000
Выгружаю 1000 пользователей с offset = 18000
Выгружаю 1000 пользователей с offset = 19000
Выгружаю 1000 пользователей с offset = 20000
Выгружаю 1000 пользователей с offset = 21000
Выгружаю 1000 пользоват

In [63]:
user_ids[-1]

6231642

# Ограничение по частоте запросов

Ограничение по частоте запросов
В API частот добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали  сервер. Подобное ограничение есть и у ВКонтакте, в документации которой указано, что можно делать не более 3-х запросов в секунду. 

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, т.к. периодичность отправки запросов будет искусственно замедлена. Воспользуемся библиотекой time и методом sleep (пауза в 0.5 секунды добавлена в конце цикла):

In [65]:
import time  
  
count = 1000  
offset = 0  
user_ids = []  
while offset < 10000:  
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    params = {  
        'group_id': 'vk',  
        'v': 5.95,  
        'count': count,  
        'offset': offset,  
        'access_token': token  
    }    
    # такой же запрос, как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']    
  
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  
    
    print('Ожидаю 0.5 секунды...')  
    time.sleep(0.5)  

Выгружаю 1000 пользователей с offset = 0
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 1000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 2000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 3000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 4000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 5000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 6000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 7000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 8000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 9000
Ожидаю 0.5 секунды...


# Лайки, репосты и комментарии

Лайки, репосты и комментарии
Ещё одна полезная вещь, которую можно получить,  —  это количество взаимодействий с постами через API новостной ленты. На данном этапе будем использовать самый простой вариант: берем последние 10 постов группы. Мы продолжаем работать с группой https://vk.com/vk.

Начнем с  формирования запроса к API ВКонтакте методом wall.get():

In [110]:
import requests  
from pprint import pprint  
  
url = 'https://api.vk.com/method/wall.get'  
params = {  
    'domain': 'vk',  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': token,  
    'v': 5.95  
}  
response = requests.get(url, params = params)  
response.json()

{'response': {'count': 430,
  'items': [{'id': 1167538,
    'from_id': -22822305,
    'owner_id': -22822305,
    'date': 1617262100,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': '',
    'copy_history': [{'id': 362,
      'owner_id': 92933,
      'from_id': 92933,
      'date': 1617261054,
      'post_type': 'post',
      'text': 'Наша команда любит шутить на 1 апреля. Каждый год мы делаем на сайте или в приложении какие-нибудь забавные мелочи. В этот раз поднимем настроение абсолютно серьёзной новостью: спустя пять лет ВКонтакте обновляет приложение для iPad. Это не шутка! \n\nМы видели, с каким энтузиазмом многие ждали этого момента, ожидание даже стало мемом. И хотя планшетами пользуются несоизмеримо реже, чем смартфонами или ноутбуками, мы всегда учитываем отзывы и пожелания, которые получаем. Это помогает запускать действительно востребованные продукты. Обновлённое приложение поможет общаться и решать повседневные вопросы — будь то быт, работа или учёба — с любого у

Видим, что сначала идёт общее количество постов, а по ключу 'items'  —  сами посты. Посмотрим на отдельный пост:

In [68]:
response.json()['response']['items'][0]  

{'id': 1167538,
 'from_id': -22822305,
 'owner_id': -22822305,
 'date': 1617262100,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'copy_history': [{'id': 362,
   'owner_id': 92933,
   'from_id': 92933,
   'date': 1617261054,
   'post_type': 'post',
   'text': 'Наша команда любит шутить на 1 апреля. Каждый год мы делаем на сайте или в приложении какие-нибудь забавные мелочи. В этот раз поднимем настроение абсолютно серьёзной новостью: спустя пять лет ВКонтакте обновляет приложение для iPad. Это не шутка! \n\nМы видели, с каким энтузиазмом многие ждали этого момента, ожидание даже стало мемом. И хотя планшетами пользуются несоизмеримо реже, чем смартфонами или ноутбуками, мы всегда учитываем отзывы и пожелания, которые получаем. Это помогает запускать действительно востребованные продукты. Обновлённое приложение поможет общаться и решать повседневные вопросы — будь то быт, работа или учёба — с любого устройства.\n\nВ прошлом году мы масштабно обновили приложение на смартфонах 

Нужная нам статистика находится в полях 'comments', 'likes' и 'reposts'. Соберем итоговую статистику для каждого поста в словарь stats. В качестве ключа будем использовать начало статьи, в качестве значения — список с тремя интересующими нас метриками и временем публикации: [комментарии, лайки, репосты, дата публикации]

In [69]:
stats = {}  
              
for record in response.json()['response']['items'][:]:  
    title = record['text'][:30]  
    if title:  
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date'] ]  
pprint(stats)  

{'Благодаря форс-мажорному 2020-': [6091, 2599, 683, 1608903643],
 'В наступающий год — со свежим ': [3245, 4927, 1716, 1608033618],
 'В этом году у нас было много т': [267, 1817, 203, 1607943610],
 'Вот уже четвёртый год мы подде': [3106, 4430, 698, 1608897114],
 'Встречайте музыкальных куратор': [2097, 2715, 550, 1603796912],
 'Мы решили не ждать курантов и ': [965, 2878, 257, 1607689227],
 'Сообщества были и остаются одн': [767, 2268, 322, 1608105691],
 'Что поможет найти лучший автос': [1127, 13471, 10209, 1608548489]}


In [137]:
def get_smm_index(group_name, token):
    smm = 0
    smm_index = None
    url_wall = 'https://api.vk.com/method/wall.get'
    params_wall = {
        'domain': group_name,
        'filter': 'owner',
        'count': 10,
        'offset': 0,
        'access_token': token,
        'v': 5.95
    }
    
    url_members = 'https://api.vk.com/method/groups.getMembers'
    params_members = {  
        'group_id': 'idealnyi_kot',  
        'v': 5.95,  
        'access_token': token  
    }     
    response_members = requests.get(url_members, params=params_members)
    members_count = response_members.json()['response']['count']
    
    response_wall = requests.get(url_wall, params=params_wall)
    
    
    for item in response_wall.json()['response']['items']:
        likes = item['likes']['count']
        comments = item['comments']['count']
        reposts = item['reposts']['count']
        smm = likes + comments + reposts
    smm_index = smm / members_count
    return smm_index

In [138]:
url_members = 'https://api.vk.com/method/groups.getMembers'
params_members = {  
    'group_id': 'idealnyi_kot',  
    'v': 5.95,  
    'access_token': token  
}     
response_members = requests.get(url_members, params=params_members)
members_count = response_members.json()['response']['count']

In [136]:
response_members.json()['response']['count']

83648

In [140]:
get_smm_index('vk', '669f554d669f554d669f554d0366e89c596669f669f554d06070c11afa2a9471230bd66')

0.0641027173718125

In [114]:
for i in response.json()['response']['items']:
    print(i)

KeyError: 'count'

In [115]:
response.json()

{'response': {'count': 430,
  'items': [{'id': 1167538,
    'from_id': -22822305,
    'owner_id': -22822305,
    'date': 1617262100,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': '',
    'copy_history': [{'id': 362,
      'owner_id': 92933,
      'from_id': 92933,
      'date': 1617261054,
      'post_type': 'post',
      'text': 'Наша команда любит шутить на 1 апреля. Каждый год мы делаем на сайте или в приложении какие-нибудь забавные мелочи. В этот раз поднимем настроение абсолютно серьёзной новостью: спустя пять лет ВКонтакте обновляет приложение для iPad. Это не шутка! \n\nМы видели, с каким энтузиазмом многие ждали этого момента, ожидание даже стало мемом. И хотя планшетами пользуются несоизмеримо реже, чем смартфонами или ноутбуками, мы всегда учитываем отзывы и пожелания, которые получаем. Это помогает запускать действительно востребованные продукты. Обновлённое приложение поможет общаться и решать повседневные вопросы — будь то быт, работа или учёба — с любого у